该部分将对基本的几何变换进行学习，几何变换的原理大多都是相似，只是变换矩阵不同，因此，我们以最常用的平移和旋转为例进行学习。在深度学习领域，我们常用平移、旋转、镜像等操作进行数据增广；在传统CV领域，由于某些拍摄角度的问题，我们需要对图像进行矫正处理，而几何变换正是这个处理过程的基础  
### 算法理论介绍
#### 变换形式
先看第一个问题，变换的形式。与OpencV不同的是这里采取冈萨雷斯的《数字图像处理_第三版》的变换矩阵方式，关于OpenCV的策略可以看它的官方文档。根据冈萨雷斯书中的描述，仿射变换的一般形式如下：  
![](https://camo.githubusercontent.com/68aa5dc26b1fd416e7f45bd1f83b1a53f8686de1/68747470733a2f2f696d672d626c6f672e6373646e696d672e636e2f32303230303431333030303235373239352e706e67)
式中的T就是变换矩阵，其中 (v,w)为原坐标，(x,y) 为变换后的坐标，不同的变换对应不同的矩阵，这里也贴出来吧，一些常见的变换矩阵及作用如下表：  
![](https://camo.githubusercontent.com/28470c13be909cd47db43e5849c4303da996fff6/68747470733a2f2f696d672d626c6f672e6373646e696d672e636e2f32303230303431333030303333343136382e706e673f782d6f73732d70726f636573733d696d6167652f77617465726d61726b2c747970655f5a6d46755a33706f5a57356e6147567064476b2c736861646f775f31302c746578745f6148523063484d364c7939696247396e4c6d4e7a5a473475626d56304c33646c61586870626c38304d4459304e7a67784f513d3d2c73697a655f31362c636f6c6f725f4646464646462c745f3730)
也就是说，我们根据自己的目的选择不同变换矩阵就可以了。
#### 坐标系变换
再看第二个问题，变换中心，对于缩放、平移可以以图像坐标原点（图像左上角为原点）为中心变换，这不用坐标系变换，直接按照一般形式计算即可。而对于旋转和偏移，一般是以图像中心为原点，那么这就涉及坐标系转换了。

我们都知道，图像坐标的原点在图像左上角，水平向右为 X 轴，垂直向下为 Y 轴。数学课本中常见的坐标系是以图像中心为原点，水平向右为 X 轴，垂直向上为 Y 轴，称为笛卡尔坐标系。看下图:
![](https://camo.githubusercontent.com/14b7f2da369d9bb4d43dcade43c151bbc1b98ec9/68747470733a2f2f696d672d626c6f672e6373646e696d672e636e2f32303230303431333030303533333434392e706e67)
因此，对于旋转和偏移，就需要3步（3次变换）：
* 将输入原图图像坐标转换为笛卡尔坐标系
* 进行旋转计算。旋转矩阵前面已经给出了
* 将旋转后的图像的笛卡尔坐标转回图像坐标
#### 图像坐标系与笛卡尔坐标系转换关系：
![](https://camo.githubusercontent.com/bfd04705020dbecca257e2ef4d11d26ce8c2ff6e/68747470733a2f2f696d672d626c6f672e6373646e696d672e636e2f32303230303431333030323932303238372e706e673f782d6f73732d70726f636573733d696d6167652f77617465726d61726b2c747970655f5a6d46755a33706f5a57356e6147567064476b2c736861646f775f31302c746578745f6148523063484d364c7939696247396e4c6d4e7a5a473475626d56304c33646c61586870626c38304d4459304e7a67784f513d3d2c73697a655f31362c636f6c6f725f4646464646462c745f3730)
在图像中我们的坐标系通常是AB和AC方向的,原点为A，而笛卡尔直角坐标系是DE和DF方向的，原点为D。 令图像表示为M×N的矩阵，对于点A而言，两坐标系中的坐标分别是(0，0)和(-N/2,M/2)，则图像某像素点(x',y')转换为笛卡尔坐标（x,y）转换关系为，x为列，y为行：  
![](https://camo.githubusercontent.com/3bae9b1c4b524bc9bd124d19ddff58b6990af05a/68747470733a2f2f696d672d626c6f672e6373646e696d672e636e2f32303230303431333030343630303531312e706e67)
逆变换为：  
![](https://camo.githubusercontent.com/d76b89d669fbd703116e1048da33368a40a82782/68747470733a2f2f696d672d626c6f672e6373646e696d672e636e2f323032303034313330303435333736372e706e67)
于是，根据前面说的3个步骤（3次变换），旋转(顺时针旋转)的变换形式就为，3次变换就有3个矩阵：  
![](https://camo.githubusercontent.com/7e57c46f23c64bbf2c6fb6096e051c8dc2e37e5a/68747470733a2f2f696d672d626c6f672e6373646e696d672e636e2f32303230303431333030353431313233332e706e67)
#### 方向映射
看第3个问题，在冈萨雷斯的《数字图像处理_第三版》中说的很清楚，前向映射就是根据原图用变换公式直接算出输出图像相应像素的空间位置，那么这会导致一个问题：可能会有多个像素坐标映射到输出图像的同一位置，也可能输出图像的某些位置完全没有相应的输入图像像素与它匹配，也就是没有被映射到，造成有规律的空洞（黑色的蜂窝状）。更好的一种方式是采用 反向映射（Inverse Mapping）：扫描输出图像的位置(x,y)，通过![](https://camo.githubusercontent.com/c2fa8e90e2f2349a52800a9613a35fbb74aedf71/68747470733a2f2f696d672d626c6f672e6373646e696d672e636e2f32303230303431333030353234373437372e706e67)（为T的逆矩阵）计算输入图像对应的位置 (v,w)，通过插值方法决定输出图像该位置的灰度值。  
#### 插值
第4个问题，采用反向映射后，需通过插值方法决定输出图像该位置的值，因此需要选择插值算法。通常有最近邻插值、双线性插值，双三次插值等，OpencV默认采用双线性插值，我们也就采用双线性插值。

#### 目前vs和opencv还没装好，暂时没实验